## Train a model and predict activities

The file works you through training a model to detect activities of a given device. The activities can be defined any action a device may allow users to do, and each activity should contain at least 3 repeated experiments to make representative learnings. 

**Before you go ahead, download the required pcap files** from [Google Drive > iot-model.tgz](https://drive.google.com/open?id=1lMqZ5qx6ATqIIiLOdTYcSm6RliK1F7vA) (size=~127MB), and decompress it to the current folder. You should expect the file structure to be `traffic/us/yi-camera/{activity_name}/{datetime}.{length}.pcap`.

**IMPORTANT** Make sure to use `python3` and install all the dependencies. 
- `pip install -r requirements.txt`


#### Extract pcap files to per-flow level info 
(output too long, hidden from this file)

In [ ]:
!./raw2intermediate.sh list_exp.txt tagged-intermediate/us

#### Parse per-flow info to features per-activity
(output too long, hidden from this file)

In [ ]:
!python extract_tbp_features.py

#### Train the model using the features
(re-reun the command below will skip the trained model, delete .model and .label.txt file to re-train) 

In [3]:
!python3 train_rf_models.py

/Users/jingjing/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Scanning features/us/yi-camera.csv
  #Data points: 2490 
	Variable: spanOfGroup          Importance: 0.409
	Variable: q90                  Importance: 0.067
	Variable: meanTBP              Importance: 0.065
	Variable: kurtosisLength       Importance: 0.048
	Variable: q60                  Importance: 0.048
	Variable: q80                  Importance: 0.048
	Variable: meanBytes            Importance: 0.042
	Variable: q70                  Importance: 0.039
	Variable: medAbsDev            Importance: 0.036
	Variable: q40                  Importance: 0.035
	Variable: skewLength           Importance: 0.03
	Variable: varTBP               Importance: 0.026
	Variable: q50                  Importance: 0.024
	Variable: medianTBP    

#### Predict activities given a pcap file

`Usage: ./predict.sh device_name path-to-pcap result-file modeldir
    Note that a temprary file  /tmp/{md5}.txt will be created during the process
    Requires python3`

In [4]:
!./predict.sh yi-camera sample-yi-camera-recording.pcap sample-result.csv tagged-models/us/

/tmp/3b986fce33742216b5f4d8d2e427766d.txt
/tmp/3b986fce33742216b5f4d8d2e427766d.txt exists, delete it to re-parse!
/Users/jingjing/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
tagged-models/us//yi-camera.model
Total packets: 1621
Number of slices: 2
Result:
             ts        ts_end  ts_delta  num_pkt              state
0  1.556329e+09  1.556329e+09  0.000019     1620  android_lan_watch
Results saved to sample-result.csv


In [5]:
!cat sample-result.csv

ts,ts_end,ts_delta,num_pkt,state,device
1556329377.198794,1556329407.828307,1.9e-05,1620,android_lan_watch,yi-camera


Explanation - between epoch time 1556329377.198794 and 1556329407.828307, the network traffic from yi-camera was predicted as the same activity as android_lan_watch, which is using android companion app to watch the video from the camera when both devices are connected to the same WI-FI network.